In [54]:
# LeNet-5 Training sur AMHCD (Tifinagh) - NumPy uniquement

import numpy as np
import os
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

# ------------------ Fonctions de base ------------------
def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

def softmax(x):
    x_stable = x - np.max(x)
    e_x = np.exp(x_stable)
    return e_x / np.sum(e_x)

def cross_entropy(y_pred, y_true):
    return -np.log(y_pred[y_true] + 1e-15)

def cross_entropy_grad(y_pred, y_true):
    grad = y_pred.copy()
    grad[y_true] -= 1.0
    return grad

# ------------------ Chargement des données ------------------
def load_amhcd_from_folder(base_path, img_size=(32, 32), normalize=True):
    class_folders = sorted(
        [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    )
    X, y = [], []
    label_map = {name: idx for idx, name in enumerate(class_folders)}
    
    for folder_name in class_folders:
        folder_path = os.path.join(base_path, folder_name)
        for fname in os.listdir(folder_path):
            if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(folder_path, fname)
                try:
                    img = Image.open(img_path).convert('L').resize(img_size)
                    img_array = np.asarray(img, dtype=np.float32)
                    if normalize:
                        img_array /= 255.0
                    X.append(img_array)
                    y.append(label_map[folder_name])
                except Exception as e:
                    print(f"Erreur avec {img_path}: {e}")
    return np.array(X).reshape(-1, 1, *img_size), np.array(y), label_map

# ------------------ Couches ------------------
class Conv2D:
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1, padding=0, lr=0.01):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.lr = lr
        scale = np.sqrt(2. / (in_channels * kernel_size * kernel_size))
        self.weights = np.random.randn(out_channels, in_channels, kernel_size, kernel_size) * scale
        self.bias = np.zeros(out_channels)

    def forward(self, x):
        self.input = x
        B, C, H, W = x.shape
        out_H = (H + 2*self.padding - self.kernel_size)//self.stride + 1
        out_W = (W + 2*self.padding - self.kernel_size)//self.stride + 1
        out = np.zeros((B, self.weights.shape[0], out_H, out_W))
        if self.padding > 0:
            x_padded = np.pad(x, ((0,0), (0,0), (self.padding,self.padding), (self.padding,self.padding)), mode='constant')
        else:
            x_padded = x
        for b in range(B):
            for c in range(self.weights.shape[0]):
                for i in range(out_H):
                    for j in range(out_W):
                        region = x_padded[b, :, i*self.stride:i*self.stride+self.kernel_size, j*self.stride:j*self.stride+self.kernel_size]
                        out[b, c, i, j] = np.sum(region * self.weights[c]) + self.bias[c]
        return out

class AveragePooling2D:
    def __init__(self, size=2, stride=2):
        self.size = size
        self.stride = stride

    def forward(self, x):
        self.input = x
        B, C, H, W = x.shape
        out_H = (H - self.size) // self.stride + 1
        out_W = (W - self.size) // self.stride + 1
        out = np.zeros((B, C, out_H, out_W))
        for b in range(B):
            for c in range(C):
                for i in range(out_H):
                    for j in range(out_W):
                        region = x[b, c, i*self.stride:i*self.stride+self.size, j*self.stride:j*self.stride+self.size]
                        out[b, c, i, j] = np.mean(region)
        return out

    def backward(self, grad_out):
        B, C, H, W = self.input.shape
        dx = np.zeros_like(self.input)
        out_H, out_W = grad_out.shape[2:]
        for b in range(B):
            for c in range(C):
                for i in range(out_H):
                    for j in range(out_W):
                        dx[b, c, i*self.stride:i*self.stride+self.size, j*self.stride:j*self.stride+self.size] += grad_out[b, c, i, j] / (self.size**2)
        return dx

class Dense:
    def __init__(self, in_features, out_features, lr=0.01):
        scale = np.sqrt(2.0 / in_features)
        self.W = np.random.randn(in_features, out_features) * scale
        self.b = np.zeros(out_features)
        self.lr = lr

    def forward(self, x):
        self.input = x
        return np.dot(x, self.W) + self.b

    def backward(self, grad_output):
    # S’assurer que tout est 2D
        if grad_output.ndim == 1:
            grad_output = grad_output[np.newaxis, :]  # (1, out)
        if self.input.ndim == 1:
            input_reshaped = self.input[np.newaxis, :]  # (1, in)
        else:
            input_reshaped = self.input

        grad_input = np.dot(grad_output, self.W.T)
        grad_W = np.dot(input_reshaped.T, grad_output)
        grad_b = np.sum(grad_output, axis=0)

        self.W -= self.lr * grad_W
        self.b -= self.lr * grad_b

        return grad_input


# ------------------ LeNet-5 ------------------
class LeNet5:
    def __init__(self, lr=0.001, num_classes=23):
        self.lr = lr
        self.c1 = Conv2D(1, 6, 5, lr=lr)
        self.s2 = AveragePooling2D(2, 2)
        self.c3 = Conv2D(6, 16, 5, lr=lr)
        self.s4 = AveragePooling2D(2, 2)
        self.fc5 = Dense(16*5*5, 120, lr)
        self.fc6 = Dense(120, 84, lr)
        self.fc7 = Dense(84, num_classes, lr)
    def forward(self, x):
        self.x1 = self.c1.forward(x)
        self.conv1_feature = self.x1.copy()
        self.x2 = relu(self.x1)
        self.x3 = self.s2.forward(self.x2)
        self.x4 = self.c3.forward(self.x3)
        self.x5 = relu(self.x4)
        self.x6 = self.s4.forward(self.x5)
        self.x7 = self.x6.reshape(self.x6.shape[0], -1)
        self.x8 = self.fc5.forward(self.x7)
        self.x9 = relu(self.x8)
        self.x10 = self.fc6.forward(self.x9)
        self.x11 = relu(self.x10)
        self.x12 = self.fc7.forward(self.x11)
        return softmax(self.x12)

    def backward(self, grad):
        grad = self.fc7.backward(grad)
        grad = relu_deriv(self.x10) * grad
        grad = self.fc6.backward(grad)
        grad = relu_deriv(self.x8) * grad
        grad = self.fc5.backward(grad)
        grad = grad.reshape(-1, 16, 5, 5)
        grad = self.s4.backward(grad)
        grad = relu_deriv(self.x4) * grad
        grad = self.c3.backward(grad)
        grad = self.s2.backward(grad)
        grad = relu_deriv(self.x1) * grad
        self.c1.backward(grad)

    def backward(self, grad):
    
        # Passe arrière à travers fc7
        grad = self.fc7.backward(grad)  # Doit retourner (1,84)
    
        # Vérification des dimensions
        if grad.shape != (1, 120):
            # Si nécessaire, ajoutez une projection linéaire
            grad = grad @ np.eye(120, 84)  # Transformation de (1,84) à (1,120)
    
        # Application de la dérivée ReLU
        grad = relu_deriv(self.fc6.input) * grad  # Maintenant (1,120) * (1,120)
    
        # Passe arrière à travers fc6
        grad = self.fc6.backward(grad)
    
        # Continuez avec les autres couches...
        grad = relu_deriv(self.fc5.input) * grad
        grad = self.fc5.backward(grad)
    
        return grad


# ------------------ Entraînement ------------------
data_dir = "amhcd-data-64/tifinagh-images"
X, y, label_map = load_amhcd_from_folder(data_dir)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

model = LeNet5(lr=0.001)
n_epochs = 10
train_losses, train_accs, val_accs = [], [], []

for epoch in range(n_epochs):
    epoch_loss = 0
    correct = 0
    for x, y_true in zip(X_train, y_train):
        x = x[np.newaxis, :, :, :]
        out = model.forward(x)
        loss = cross_entropy(out[0], y_true)
        grad = cross_entropy_grad(out[0], y_true)
        model.backward(grad)
        epoch_loss += loss
        correct += (np.argmax(out[0]) == y_true)

    train_loss = epoch_loss / len(X_train)
    train_acc = correct / len(X_train)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    val_correct = 0
    y_preds, y_trues = [], []
    for x, y_true in zip(X_val, y_val):
        x = x[np.newaxis, :, :, :]
        out = model.forward(x)
        pred = np.argmax(out[0])
        y_preds.append(pred)
        y_trues.append(y_true)
        val_correct += (pred == y_true)
    val_acc = val_correct / len(X_val)
    val_accs.append(val_acc)

    print(f"Epoch {epoch+1}/{n_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2%} | Val Acc: {val_acc:.2%}")

# ------------------ Visualisation ------------------
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train Acc')
plt.plot(val_accs, label='Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

cm = confusion_matrix(y_trues, y_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

features = model.conv1_feature[0]
plt.figure(figsize=(12, 6))
for i in range(6):
    plt.subplot(2, 3, i+1)
    plt.imshow(features[i], cmap='gray')
    plt.title(f'Feature Map {i+1}')
    plt.axis('off')
plt.suptitle('Feature Maps from First Conv Layer')
plt.tight_layout()
plt.show()


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 120 is different from 84)